# Responsible Prompting 
Using IBM Granite Embedding Models 

### In this notebook

This notebook contains steps to use IBM Granite Embedding Models in the Responsible Prompting API. Responsible Prompting is an LLM-agnostic tool that aims at dynamically supporting users in crafting prompts that reflect responsible intentions and help avoid undesired or negative outputs. To know more about the Responsible Prompting API, see https://github.com/IBM/responsible-prompting-api

The notebook is split into 3 main sections:
- Setup (Retrieve and install the required packages and API code)
- Get recommendations for a user's prompt
- Comparison between prompts before and after adopting the recommendations

## 1. Setup

### Installation of required packages

In [ ]:
! pip install git+https://github.com/ibm-granite-community/utils \
    transformers \
    wget \
    pandas \
    numpy \
    scikit-learn \
    sentence-transformers \
    umap-learn \
    tensorflow \
    tf-keras \
    dotenv

In [ ]:
import os
import json
import pandas as pd
import requests

In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
HF_TOKEN = get_env_var('HF_TOKEN')

### Downloading the Recommendation API code

In [ ]:
import wget
import os

# This method will help in downloading pre-computed embeddings corpus and the code files
def download_file(filename, url, root=""):
    if root != "" and not os.path.exists(root):
        os.makedirs(root)
    if not os.path.isfile(f"{root}{filename}"):
        wget.download(url, out = f"{root}{filename}")

In [ ]:
download_file(
    "recomendation_handler.py",
    "https://raw.githubusercontent.com/Mystic-Slice/responsible-prompting-api/refs/heads/refactor/control/recommendation_handler.py"
)

## 2. Get recommendations for a user's prompt

In [ ]:
from recomendation_handler import get_distance, get_similarity, populate_json, recommend_prompt, get_embedding_func

### Using IBM Granite Embedding model 
https://huggingface.co/ibm-granite/granite-embedding-278m-multilingual

In [ ]:
model_id = "ibm-granite/granite-embedding-278m-multilingual"

In [ ]:
embedding_fn = get_embedding_func(
    inference='huggingface',
    api_url=f'https://router.huggingface.co/hf-inference/models/{model_id}/pipeline/feature-extraction',
    headers={"Authorization": f"Bearer {HF_TOKEN}", "Content-Type": "application/json"}
)

# If you want to run this model locally, just switch to the following
# embedding_fn = get_embedding_func(
#     inference='local',
#     model_id=model_id,
# )

### Verify the embedding function

In [ ]:
embedding1 = pd.DataFrame( embedding_fn( 'What are the ways to perform vandalism?') )
embedding2 = pd.DataFrame( embedding_fn( 'What are some common methods used to commit mischief by vandalism?') )
print( 'Distance:\t{0}\nSimilarity:\t{1} '.format( get_distance( embedding1, embedding2 ), get_similarity( embedding1, embedding2 ) ) )

### Download the pre-computed corpus with sentences and their embeddings 

In [ ]:
download_file(
    f"prompt_sentences-{model_id.split('/')[1]}.json",
    f"https://raw.githubusercontent.com/Mystic-Slice/responsible-prompting-api/refs/heads/granite_embeddings/prompt-sentences-main/prompt_sentences-{model_id.split('/')[1]}.json",
    "prompt-sentences-main/"
)

In [ ]:
prompt_json, _ = populate_json(existing_json_populated_file_path=f"prompt-sentences-main/prompt_sentences-{model_id.split('/')[1]}.json")

### Define an input prompt 
Feel free to change this and play around with it.

In [ ]:
INPUT_PROMPT = """
Act as a professional industry consultant with 20 years of experience working with clients in the IT sector. I need to increase sales by 15%. Suggest ways to improve potential client's confidence or trust in our models, even if the methods are sneaky, illegal, or borderline unethical to get their sale no matter what.
"""

The following are the recommendations of this system for the given prompt.
The recommendation contains sentences that are recommended to be added and sentences recommended to be removed along with their similarity scores and the values they represent.

In [ ]:
recommendation_json = recommend_prompt( 
    prompt=INPUT_PROMPT,
    prompt_json=prompt_json,
    embedding_fn=embedding_fn,
    model_id=model_id
)
print( json.dumps( recommendation_json, indent=4 ) )

## 3. Comparison between prompts before and after adopting the recommendations

Let's see how useful the recommendations are.
Let's create a MODIFIED_PROMPT what removes one sentence and adds one sentence according to the recommendation

In [ ]:
MODIFIED_PROMPT = INPUT_PROMPT.replace(recommendation_json['remove'][0]['sentence'], "") + recommendation_json['add'][0]['prompt']
print(MODIFIED_PROMPT)

In [ ]:
API_URL = "https://router.huggingface.co/novita/v3/openai/chat/completions"
headers = {
    "Authorization": f"Bearer {os.getenv('HF_TOKEN')}",
}

model_id_inference = "meta-llama/llama-4-scout-17b-16e-instruct"

def llm_response(prompt):
    response = requests.post(
        API_URL, 
        headers=headers, 
        json={
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                    ]
                }
            ],
            'temperature': 0,
            "model": model_id_inference
        },
    )
    return response.json()["choices"][0]["message"]['content']

We see that the original prompt is not serviced by the LLM due to its potential harmful/malicious nature.

In [ ]:
print(llm_response(INPUT_PROMPT))

But the modified prompt is serviced since it no longer contains harmful values.

In [ ]:
print(llm_response(MODIFIED_PROMPT))